<a href="https://colab.research.google.com/github/KnoBuddy/Aviato/blob/main/Erlich_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### KnoBuddy's Colab Notebook Implementation of Erlich a text2image generator for logos

Connect your google drive and create a folder name test_nb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun  9 21:03:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
%cd drive/MyDrive/test_nb/

/content/drive/MyDrive/test_nb


In [ ]:
! ls

'=1.0.8'		     latent-diffusion
'=2.0.0'		     ldm
 aesthetic_clip_embeds	     LICENSE
 aesthetic-predictor	     ongo.pt
 autoedit_api.py	     outputs_finetune_0_18144__0_25.0
 autoedit.py		     outputs_finetune_0_18144__0_25.0_npy
 bert.pt		     outputs_finetune_0_95838__0_25.0
 clip_custom		     outputs_finetune_0_95838__0_25.0_npy
 cog_predict.py		     predict_util.py
 cog.yaml		     README.md
 diffusion.pt		     requirements.txt
 encoders		     sample.py
 finetune.pt		     scripts
 guided_diffusion	     setup.py
 guided_diffusion.egg-info   src
 inpaint.pt		     taming-transformers
 kl-f8.pt		     wandb


In [ ]:
! wget -O erlich.pt https://huggingface.co/laion/erlich/raw/main/model/ema_0.9999_120000.pt

--2022-06-10 00:59:15--  https://huggingface.co/laion/erlich/raw/main/model/ema_0.9999_120000.pt
Resolving huggingface.co (huggingface.co)... 18.204.39.151, 34.198.184.113, 34.197.58.156, ...
Connecting to huggingface.co (huggingface.co)|18.204.39.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135 [text/plain]
Saving to: ‘erlich.pt’

erlich.pt           100%[===================>]     135  --.-KB/s    in 0s      

2022-06-10 00:59:16 (27.0 MB/s) - ‘erlich.pt’ saved [135/135]



In [ ]:
! git clone https://github.com/LAION-AI/ldm-finetune

Cloning into 'ldm-finetune'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 292 (delta 134), reused 265 (delta 107), pack-reused 0
Receiving objects: 100% (292/292), 1.95 MiB | 12.83 MiB/s, done.
Resolving deltas: 100% (134/134), done.


Use this to update

In [ ]:
! git pull

^C


In [ ]:
%cd ldm-finetune/

/content/drive/MyDrive/test_nb/ldm-finetune


In [ ]:
! wandb login 02a7247d36c07136ba7b90e08d6201ee8e67bc1b

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
! git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
! git pull

Already up to date.


In [ ]:

! sudo git clone https://huggingface.co/laion/erlich

Cloning into 'erlich'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.
Filtering content: 100% (6/6), 17.27 GiB | 27.69 MiB/s, done.
fatal: cannot exec '.git/hooks/post-checkout': Permission denied
Encountered %!d(MISSING) file(s) that may not have been copied correctly on Windows:
	optimizer/opt080000.pt
	optimizer/opt120000.pt
	optimizer/opt040000.pt

See: `git lfs help smudge` for more details.


In [ ]:
! git lfs help smudge

Sorry, no usage text found for "git-lfs"


In [ ]:
! git clone https://github.com/LAION-AI/aesthetic-predictor

fatal: destination path 'aesthetic-predictor' already exists and is not an empty directory.


In [ ]:
! git clone https://github.com/CompVis/latent-diffusion.git
! git clone https://github.com/CompVis/taming-transformers
! pip install -e ./taming-transformers
! pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops

In [ ]:
! pip install -e .

In [ ]:
! pip install -r requirements.txt
! pip install ftfy

In [ ]:

! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-h_j328oo
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-h_j328oo


In [ ]:
# text encoder (required)
! wget https://dall-3.com/models/glid-3-xl/bert.pt

# ldm first stage (required)
! wget https://dall-3.com/models/glid-3-xl/kl-f8.pt

# there are several diffusion models to choose from:

# original diffusion model from CompVis
! wget https://dall-3.com/models/glid-3-xl/diffusion.pt

# new model fine tuned on a cleaner dataset (will not generate watermarks, split images or blurry images)
! wget https://dall-3.com/models/glid-3-xl/finetune.pt

# inpaint
! wget https://dall-3.com/models/glid-3-xl/inpaint.pt

# erlich
! wget -O erlich.pt https://huggingface.co/laion/erlich/raw/main/model/ema_0.9999_120000.pt

# ongo
! wget https://huggingface.co/laion/ongo/resolve/main/ongo.pt

In [ ]:
! git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
# generated images saved to ./output/
# generated image embeddings saved to ./output_npy/ as npy files

In [ ]:
# fast PLMS sampling
! python sample.py --model_path finetune.pt --batch_size 1 --num_batches 1 --text "a cyberpunk girl with a scifi neuralink device on her head"

Using device: cuda
tcmalloc: large alloc 3491364864 bytes == 0x7c08000 @  0x7f3f2f080b6b 0x7f3f2f0a0379 0x7f3e74bad50e 0x7f3e74b9f7c2 0x7f3eb02294c1 0x7f3f2a1f6446 0x7f3f29ecd3f1 0x593784 0x594731 0x548cc1 0x51566f 0x549e0e 0x593fce 0x5118f8 0x549e0e 0x4bcb19 0x59582d 0x595b69 0x62026d 0x55de15 0x59af67 0x515655 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173
wandb: Currently logged in as: knobuddy. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in /content/drive/MyDrive/test_nb/ldm-finetune/wandb/run-20220609_211358-x9851z57
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run zany-fire-35
wandb: ⭐️ View project at https://wandb.ai/knobuddy/ongo-eval-table
wandb: 🚀 View run at https://wandb.ai/knobuddy/ongo-eval-table/runs/x9851z57
Starting run for	a cyberpu

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --batch_size 1 --num_batches 12 --text "a cyberpunk girl with a scifi neuralink device on her head | trending on artstation"

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --batch_size 2 --num_batches 1 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group"

Using device: cuda
tcmalloc: large alloc 3491364864 bytes == 0x7258000 @  0x7fc4d81bab6b 0x7fc4d81da379 0x7fc41dce750e 0x7fc41dcd97c2 0x7fc4593634c1 0x7fc4d3330446 0x7fc4d30073f1 0x593784 0x594731 0x548cc1 0x51566f 0x549e0e 0x593fce 0x5118f8 0x549e0e 0x4bcb19 0x59582d 0x595b69 0x62026d 0x55de15 0x59af67 0x515655 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173
wandb: Currently logged in as: knobuddy. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in /content/drive/MyDrive/test_nb/ldm-finetune/wandb/run-20220609_212751-3v67kozz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ruby-frost-39
wandb: ⭐️ View project at https://wandb.ai/knobuddy/ongo-eval-table
wandb: 🚀 View run at https://wandb.ai/knobuddy/ongo-eval-table/runs/3v67kozz
Starting run for	the logo

In [ ]:
# sample with an init image
!python sample.py --init_image picture.jpg --skip_timesteps 10 --model_path finetune.pt --batch_size 6 --num_batches 6 --text "a cyberpunk girl with a scifi neuralink device on her head"

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --guidance_scale 30.0 --seed -1 --batch_size 1 --num_batches 1 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group" --dropout 0.1 --ema_rate 0.9999 --attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 32 --learn_sigma False --noise_schedule linear --num_channels 320 --num_heads 8 --num_res_blocks 2 --resblock_updown False --use_fp16 True --use_scale_shift_norm False

usage: sample.py [-h] [--model_path MODEL_PATH] [--kl_path KL_PATH]
                 [--bert_path BERT_PATH] [--text TEXT] [--edit EDIT]
                 [--edit_x EDIT_X] [--edit_y EDIT_Y] [--edit_width EDIT_WIDTH]
                 [--edit_height EDIT_HEIGHT] [--mask MASK]
                 [--negative NEGATIVE] [--init_image INIT_IMAGE]
                 [--skip_timesteps SKIP_TIMESTEPS] [--prefix PREFIX]
                 [--num_batches NUM_BATCHES] [--batch_size BATCH_SIZE]
                 [--width WIDTH] [--height HEIGHT] [--seed SEED]
                 [--guidance_scale GUIDANCE_SCALE] [--steps STEPS] [--cpu]
                 [--clip_score] [--clip_guidance]
                 [--clip_guidance_scale CLIP_GUIDANCE_SCALE] [--cutn CUTN]
                 [--ddim] [--ddpm]
sample.py: error: unrecognized arguments: --dropout 0.1 --ema_rate 0.9999 --attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 32 --learn_sigma False --noise_schedule linear --num_channe

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --guidance_scale 25.0 --seed -1 --batch_size 1 --num_batches 100 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group. the name of the group KnoBuddy is spelled out across the bottom of the logo"

Using device: cuda
tcmalloc: large alloc 3491364864 bytes == 0x7832000 @  0x7fbf8f16ab6b 0x7fbf8f18a379 0x7fbed4c9750e 0x7fbed4c897c2 0x7fbf103134c1 0x7fbf8a2e0446 0x7fbf89fb73f1 0x593784 0x594731 0x548cc1 0x51566f 0x549e0e 0x593fce 0x5118f8 0x549e0e 0x4bcb19 0x59582d 0x595b69 0x62026d 0x55de15 0x59af67 0x515655 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173
wandb: Currently logged in as: knobuddy. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in /content/drive/MyDrive/test_nb/ldm-finetune/wandb/run-20220610_003721-1vl2h9yv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run super-armadillo-55
wandb: ⭐️ View project at https://wandb.ai/knobuddy/ongo-eval-table
wandb: 🚀 View run at https://wandb.ai/knobuddy/ongo-eval-table/runs/1vl2h9yv
Starting run for	the

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path erlich.pt --guidance_scale 25.0 --seed -1 --batch_size 1 --num_batches 100 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group. the name of the group KnoBuddy is spelled out across the bottom of the logo"

Using device: cuda
Traceback (most recent call last):
  File "sample.py", line 341, in <module>
    model_state_dict = torch.load(args.model_path, map_location="cpu")
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 713, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 920, in _legacy_load
    magic_number = pickle_module.load(f, **pickle_load_args)
_pickle.UnpicklingError: invalid load key, 'v'.
